[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danielmlow/llm_course/blob/main/text/sentiment_analysis_emotion_recognition.ipynb)

# Sentiment analysis and emotion classification (pre-trained)

If it's super slow you can activate the GPU in Colab under Runtime -> Change runtime type > T4 GPU

In [ ]:
!pip install torch==2.8.0
# maybe: !pip install -q transformers==4.33.2

  Using cached torch-2.8.0-cp311-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.0 kB)
Using cached torch-2.8.0-cp311-none-macosx_11_0_arm64.whl (73.6 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp311-cp311-macosx_11_0_arm64.whl (12 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [torch]32m5/6 [torch]kx]


In [5]:
import pandas as pd
import os
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

on_colab = False

if on_colab:
  from google.colab import drive
  project_name = 'project_name'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/consulting/matt/er/data/input/'
  output_dir = f'/content/drive/MyDrive/consulting/matt/er/data/output/'
else:
  input_dir = './data/input/'
  output_dir = './data/output/'

os.makedirs(output_dir, exist_ok=True)




In [6]:

# Change to your dataset:

# docs = pd.read_csv(input_dir + 'dataset.csv')['text_col'].values
# docs = [str(n) for n in docs] # make sure they'll all strings


docs = ['I am happy',
        "I'm happy, but worried about tomorrow",
        "I'm miserable",
        "I'm sad, but hopeful",
        'I am not happy',
        'I wish I were happy',
        "I'm sad, but hopeful",
        "Don't talk to me like that!",
        "Really? I'm shocked!"]

In [7]:
def huggingface_output_2_df(output_dict, add_to_col_names = None):
	feature_names = [n.get('label') for n in output_dict[0]]
	if add_to_col_names:
		feature_names = [add_to_col_names+n for n in feature_names]
	feature_vectors = []
	for doc in output_dict:
		feature_vectors_doc = []
		for feature in doc:
			feature_vectors_doc.append(feature.get('score'))
		feature_vectors.append(feature_vectors_doc)
	feature_vectors = pd.DataFrame(feature_vectors, columns = feature_names)
	return feature_vectors

# Extract sentiment analysis


In [4]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
output_dict = sentiment_pipeline(docs, return_all_scores=True)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/danielmlow/miniconda3/envs/llm_course/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/Users/danielmlow/miniconda3/envs/llm_course/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [8]:
feature_vectors = huggingface_output_2_df(output_dict, add_to_col_names = 'sentiment_')
feature_vectors['docs'] = docs
feature_vectors

,sentiment_NEGATIVE,sentiment_POSITIVE,docs
0,0.000120,0.999880,I am happy
1,0.022988,0.977012,"I'm happy, but worried about tomorrow"
2,0.999780,0.000220,I'm miserable
3,0.000404,0.999596,"I'm sad, but hopeful"
4,0.999790,0.000210,I am not happy
5,0.050676,0.949324,I wish I were happy
6,0.000404,0.999596,"I'm sad, but hopeful"
7,0.648448,0.351552,Don't talk to me like that!
8,0.018542,0.981458,Really? I'm shocked!


In [ ]:
from transformers import pipeline

sentiment_es = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis",
    tokenizer="pysentimiento/robertuito-sentiment-analysis",
)

texts = ["Me encanta esta película", "No me gustó para nada", "Está bien, pero no es espectacular."]
results = sentiment_es(texts, return_all_scores=True)




/Users/danielmlow/miniconda3/envs/llm_course/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

[[{'label': 'NEG', 'score': 0.006506713572889566}, {'label': 'NEU', 'score': 0.037519678473472595}, {'label': 'POS', 'score': 0.9559735655784607}], [{'label': 'NEG', 'score': 0.8632596135139465}, {'label': 'NEU', 'score': 0.08551418781280518}, {'label': 'POS', 'score': 0.051226191222667694}], [{'label': 'NEG', 'score': 0.21275319159030914}, {'label': 'NEU', 'score': 0.5675188302993774}, {'label': 'POS', 'score': 0.21972791850566864}]]


/Users/danielmlow/miniconda3/envs/llm_course/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [8]:
feature_vectors = huggingface_output_2_df(results, add_to_col_names = 'sentiment_es_')
feature_vectors

,sentiment_es_others,sentiment_es_joy,sentiment_es_sadness,sentiment_es_anger,sentiment_es_surprise,sentiment_es_disgust,sentiment_es_fear
0,0.002643,0.989545,0.001423,0.000484,0.003663,0.000980,0.001263
1,0.049876,0.018882,0.429593,0.270996,0.034459,0.123935,0.072258


# Emotion recognition

All models trained on Go Emotions: https://huggingface.co/models?dataset=dataset:go_emotions

Model: https://huggingface.co/SamLowe/roberta-base-go_emotions 

- Accuracy: 0.474
- Precision: 0.575
- Recall: 0.396
- F1: 0.450




In [ ]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

[{'label': 'disappointment', 'score': 0.46669477224349976}, {'label': 'sadness', 'score': 0.39849501848220825}, {'label': 'annoyance', 'score': 0.06806589663028717}, {'label': 'neutral', 'score': 0.05703022703528404}, {'label': 'disapproval', 'score': 0.04423929750919342}, {'label': 'nervousness', 'score': 0.014850744046270847}, {'label': 'realization', 'score': 0.014059885405004025}, {'label': 'approval', 'score': 0.011267449706792831}, {'label': 'joy', 'score': 0.0063033876940608025}, {'label': 'remorse', 'score': 0.006221481133252382}, {'label': 'caring', 'score': 0.006029403302818537}, {'label': 'embarrassment', 'score': 0.00526547571644187}, {'label': 'anger', 'score': 0.00498143071308732}, {'label': 'disgust', 'score': 0.004259033594280481}, {'label': 'grief', 'score': 0.004002132918685675}, {'label': 'confusion', 'score': 0.003382918192073703}, {'label': 'relief', 'score': 0.0031404944602400064}, {'label': 'desire', 'score': 0.0028274673968553543}, {'label': 'admiration', 'score

In [ ]:
model_outputs = classifier(docs)
feature_vectors = huggingface_output_2_df(model_outputs, add_to_col_names = 'roberta_base_')
feature_vectors

,roberta_base_joy,roberta_base_neutral,roberta_base_approval,roberta_base_relief,roberta_base_admiration,roberta_base_excitement,roberta_base_gratitude,roberta_base_caring,roberta_base_amusement,roberta_base_love,...,roberta_base_desire,roberta_base_disappointment,roberta_base_curiosity,roberta_base_nervousness,roberta_base_surprise,roberta_base_remorse,roberta_base_grief,roberta_base_embarrassment,roberta_base_fear,roberta_base_disgust
0,0.895246,0.035179,0.028167,0.026642,0.020664,0.016066,0.015526,0.014913,0.013467,0.012827,...,0.002370,0.002130,0.001743,0.001603,0.001498,0.001199,0.001178,0.000986,0.000782,0.000671
1,0.698884,0.123571,0.083955,0.079816,0.037634,0.032949,0.030415,0.025278,0.021513,0.017859,...,0.006882,0.006632,0.006038,0.005396,0.004900,0.004414,0.003442,0.002974,0.002872,0.001043
2,0.810011,0.176840,0.035794,0.033110,0.013010,0.012448,0.011800,0.008670,0.007642,0.006905,...,0.002742,0.002584,0.002167,0.002148,0.001678,0.001404,0.001256,0.001117,0.001059,0.000328
3,0.875564,0.160377,0.069820,0.037162,0.033953,0.029319,0.024180,0.019917,0.017625,0.013948,...,0.004674,0.004665,0.004449,0.004171,0.004107,0.003174,0.002981,0.002891,0.001834,0.000926
4,0.572092,0.288714,0.048506,0.045166,0.043421,0.012110,0.011123,0.011011,0.009509,0.009503,...,0.002971,0.002456,0.002219,0.002065,0.002063,0.001938,0.001729,0.001073,0.000625,0.000403
5,0.837150,0.053849,0.042612,0.015282,0.015040,0.012993,0.011984,0.010435,0.009940,0.009412,...,0.004523,0.004350,0.002132,0.001641,0.001582,0.001439,0.001318,0.001199,0.000982,0.000874
6,0.875564,0.160377,0.069820,0.037162,0.033953,0.029319,0.024180,0.019917,0.017625,0.013948,...,0.004674,0.004665,0.004449,0.004171,0.004107,0.003174,0.002981,0.002891,0.001834,0.000926
7,0.540450,0.391780,0.198641,0.131819,0.015648,0.009250,0.008194,0.006436,0.005205,0.003862,...,0.001634,0.001517,0.001437,0.001395,0.001064,0.000788,0.000520,0.000502,0.000489,0.000413
8,0.884367,0.048370,0.041547,0.032006,0.017881,0.016813,0.010565,0.007919,0.007185,0.007104,...,0.003627,0.003565,0.003247,0.003173,0.002059,0.001555,0.001475,0.001339,0.001301,0.000890


In [ ]:
from transformers import pipeline

emotion_es = pipeline(
    "text-classification",
    model="pysentimiento/robertuito-emotion-analysis",
    tokenizer="pysentimiento/robertuito-emotion-analysis",
    # optionally top_k=None if you want all emotion scores
)

texts = ["Estoy tan feliz hoy", "Estoy muy enfadado con lo que pasó"]
results = emotion_es(texts, return_all_scores=True)
feature_vectors

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

[[{'label': 'others', 'score': 0.0026429127901792526}, {'label': 'joy', 'score': 0.9895449280738831}, {'label': 'sadness', 'score': 0.0014225412160158157}, {'label': 'anger', 'score': 0.0004835830768570304}, {'label': 'surprise', 'score': 0.003662543836981058}, {'label': 'disgust', 'score': 0.0009804358705878258}, {'label': 'fear', 'score': 0.0012630944838747382}], [{'label': 'others', 'score': 0.04987611621618271}, {'label': 'joy', 'score': 0.01888248138129711}, {'label': 'sadness', 'score': 0.429593026638031}, {'label': 'anger', 'score': 0.27099573612213135}, {'label': 'surprise', 'score': 0.03445936739444733}, {'label': 'disgust', 'score': 0.12393507361412048}, {'label': 'fear', 'score': 0.07225818186998367}]]


In [ ]:
%%time

# if you have a lot of data, loop through every 1000 files and save in case the session dies. Then you can pick up where you left off.

feature_vectors_all = []
step = 1000
print(len(docs))
for i in range(0, len(docs), step):
  print(str(i).zfill(5))
  docs_i = docs[i:i+step]
  output_dict = emotion(docs_i, return_all_scores=True)
  feature_vectors = huggingface_output_2_df(output_dict, add_to_col_names = 'roberta_base_')
  feature_vectors['event'] = docs_i
  feature_vectors.to_csv(output_dir + f'roberta_base_{str(i).zfill(5)}.csv')
  feature_vectors_all.append(feature_vectors)


feature_vectors_all = pd.concat(feature_vectors_all).reset_index(drop=True)
feature_vectors_all.to_csv(output_dir + f'messages_emoroberta.csv') # save